In [8]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

In [11]:
# This function loads in our trained model w/ weights and compiles it 
from source.predict import  load_model

# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)   
    if (ds.Modality == 'DX') and (ds.BodyPartExamined=='CHEST') and (ds.PatientPosition in ['PA','AP']):
        img = ds.pixel_array
        return img
    else:
        print('Condition not passed: "Modality is not DX" or "BodyPartExamined is not Chest" or "PatientPosition is not either PA or AP"\n')
        return None
    
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = ((img/255)-img_mean)/img_std
    proc_img = resize(proc_img, img_size)
    return proc_img

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred = model.predict(img)
    if pred > thresh:
        prediction = 'Pneumonia'
    else:
        prediction = 'Non-pneumonia'
    
    return prediction 

In [12]:
import os
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

model_name = 'vgg16'
my_model = load_model(model_path = os.path.join('output', f'my_{model_name}.json'), 
                   weight_path =  os.path.join('output', f"xray_class_{model_name}.best.hdf5"))
my_model.thresh = 0.44090602
print("thresh:", my_model.thresh)
thresh = my_model.thresh #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    actual = pydicom.dcmread(i).StudyDescription    
    print(f'''Model predicted {pred} with thresh = {thresh:.2}, while the actual case is {actual}\n''')

thresh: 0.44090602
Load file test1.dcm ...
Model predicted Pneumonia with thresh = 0.44, while the actual case is No Finding

Load file test2.dcm ...
Model predicted Pneumonia with thresh = 0.44, while the actual case is Cardiomegaly

Load file test3.dcm ...
Model predicted Pneumonia with thresh = 0.44, while the actual case is Effusion

Load file test4.dcm ...
Condition not passed: "Modality is not DX" or "BodyPartExamined is not Chest" or "PatientPosition is not either PA or AP"

Load file test5.dcm ...
Condition not passed: "Modality is not DX" or "BodyPartExamined is not Chest" or "PatientPosition is not either PA or AP"

Load file test6.dcm ...
Condition not passed: "Modality is not DX" or "BodyPartExamined is not Chest" or "PatientPosition is not either PA or AP"

